# Objetivo

  Recolectar información de las APIS públicas de MercadoLibre (https://developers.mercadolibre.com.ar/es_ar/api-docs-es) para luego a través de un modelado simple, poder predecir, dada una publicación, la cantidad de ítems vendidos.

  El análisis se acotará a Argentina y se recolectará información sobre cámaras y accesorios, teniendo en cuenta aproximadamente 1000 publicaciones por cada una de las subcategorías presentes dentro de la categoría mencionada. 

  Finalmente, se guardará la información recolectada en un archivo en formato .csv.



# Importanciones


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd "drive/MyDrive/DataScienceChallengeMELI/Modelado"

[Errno 2] No such file or directory: 'drive/MyDrive/DataScienceChallengeMELI/Modelado'
/content/drive/MyDrive/DataScienceChallengeMELI/Modelado


In [9]:
!pip install import_ipynb

In [10]:
import numpy as np
import pandas as pd

import time
from time import process_time

import import_ipynb
from funciones_aux import request_a_diccionario, generar_dfs

importing Jupyter notebook from funciones_aux.ipynb


# Recolección de información

Se empezará analizando los paises que se vinculan a MercadoLibre.

In [11]:
lista_paises = request_a_diccionario("https://api.mercadolibre.com/sites")

In [14]:
print('Paises que tienen acceso a MercadoLibre:')
for pais in lista_paises:
  print(pais['name'])

Paises que tienen acceso a MercadoLibre:
Guatemala
Bolivia
Nicaragua
El Salvador
Ecuador
Argentina
Cuba
Costa Rica
Perú
Paraguay
Dominicana
Chile
Venezuela
Mexico
Honduras
Panamá
Uruguay
Colombia
Brasil


Se considera necesario aclarar nuevamente que la información sobre la que se trabajará será únicamente del sitio de MercadoLibre Argentina.

Se determinan las categorías presentes en el sitio.

In [15]:
lista_categorias = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/categories")

In [16]:
print("Hay un total de", len(lista_categorias), "categorías.")

Hay un total de 32 categorías.


In [17]:
print('Las categorías disponibles en el sitio son:')
for categoria in lista_categorias:
  print(categoria['name'])

Las categorías disponibles en el sitio son:
Accesorios para Vehículos
Agro
Alimentos y Bebidas
Animales y Mascotas
Antigüedades y Colecciones
Arte, Librería y Mercería
Autos, Motos y Otros
Bebés
Belleza y Cuidado Personal
Cámaras y Accesorios
Celulares y Teléfonos
Computación
Consolas y Videojuegos
Construcción
Deportes y Fitness
Electrodomésticos y Aires Ac.
Electrónica, Audio y Video
Entradas para Eventos
Herramientas
Hogar, Muebles y Jardín
Industrias y Oficinas
Inmuebles
Instrumentos Musicales
Joyas y Relojes
Juegos y Juguetes
Libros, Revistas y Comics
Música, Películas y Series
Ropa y Accesorios
Salud y Equipamiento Médico
Servicios
Souvenirs, Cotillón y Fiestas
Otras categorías


Se analizará la información perteneciente a la categoría: Cámaras y accesorios.

In [53]:
categorias_all = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/categories/all")

In [54]:
lista_subcategorias_id = []
lista_subcategorias_name = []
lista_subcategorias_cant_items = []

for categoria_id in list(categorias_all.keys()):
    dicc_info_particular = categorias_all[categoria_id]
    categoria_name = dicc_info_particular["name"]
    cant_items_totales = dicc_info_particular["total_items_in_this_category"]
    lista_categorias_padre = dicc_info_particular['path_from_root']
    categoria_principal_id = lista_categorias_padre[0]["id"]
    if categoria_principal_id == "MLA1039" and len(lista_categorias_padre) > 1:
        lista_subcategorias_id.append(categoria_id)
        lista_subcategorias_name.append(categoria_name)
        lista_subcategorias_cant_items.append(cant_items_totales)

df_info_subcategorias = pd.DataFrame({"id": lista_subcategorias_id, "nombre":lista_subcategorias_name, 
              "cant_items":lista_subcategorias_cant_items})

In [55]:
df_info_subcategorias.sort_values("cant_items", ascending = False)

,id,nombre,cant_items
5,MLA1049,Accesorios para Cámaras,32852
2,MLA430361,Cámaras,21234
28,MLA1042,Cámaras Digitales,14871
7,MLA59376,Lentes y Filtros,12385
55,MLA412430,Soportes y Estabilizadores,9487
...,...,...,...
115,MLA430415,Motores,0
114,MLA430414,Módulos,0
113,MLA430413,Mochilas y Estuches,0
112,MLA430412,Hélices,0


In [58]:
print("Hay un total de", len(df_info_subcategorias), "subcategorías de la categoría 'Cámaras y accesorios'.")

Hay un total de 136 subcategorías de la categoría 'Cámaras y accesorios'.


Se recolectará la información perteneciente a la categoría seleccionada.

In [60]:
t_inicial = time.perf_counter()

lista_dfs = []
for numero_categoria, categoria_id in enumerate(lista_subcategorias_id):
    categoria_name = lista_subcategorias_name[numero_categoria]
    cant_items_totales = lista_subcategorias_cant_items[numero_categoria]
    print("Recolectando data de la categoría:", str(numero_categoria), "de", len(lista_subcategorias_id)-1, "(categoría", categoria_name, ").")
    for pagina in range(20):
        print("Página:" + str(pagina) + " de 19.", end='\r', flush=True)
        offset = str(50 * pagina + 1)      
        items = request_a_diccionario("https://api.mercadolibre.com/sites/MLA/search?category=" + categoria_id + "&limit=50&offset=" + offset)
        df = generar_dfs(items)
        lista_dfs.append(df)
        time.sleep(1)
    elapsed_time = time.perf_counter() - t_inicial
    print("Tiempo parcial de ejecución:", round(elapsed_time / 60, 2), "minutos.")
    print("-"*100)
        
elapsed_time = time.perf_counter() - t_inicial

print("Tiempo total de ejecución:", round(elapsed_time / 60, 2), "minutos.")

Recolectando data de la categoría: 0 de 135 (categoría Álbumes y Portarretratos ).
Tiempo parcial de ejecución: 0.46 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 1 de 135 (categoría Cables ).
Tiempo parcial de ejecución: 0.89 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 2 de 135 (categoría Cámaras ).
Tiempo parcial de ejecución: 1.37 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 3 de 135 (categoría Filmadoras y Cámaras de Acción ).
Tiempo parcial de ejecución: 1.82 minutos.
----------------------------------------------------------------------------------------------------
Recolectando data de la categoría: 4 de 135 (categoría Drones y Accesorios ).
Tiempo parcial de ejecución: 2.28 minutos.


# Generación del dataset

In [63]:
df_total_original = pd.concat(lista_dfs, axis = 0)

Se eliminan los duplicados

In [64]:
print("Cantidad de registros totales:", len(df_total_original))
print("Cantidad de duplicados:", df_total_original.duplicated(keep="first").sum())
df_total_original = df_total_original[~df_total_original.duplicated(keep="first")]
print("Se eliminan los duplicados.")
print("Quedan", len(df_total_original), "registros en total.")

Cantidad de registros totales: 67900
Cantidad de duplicados: 18896
Se eliminan los duplicados.
Quedan 49004 registros en total.


Generación del archivo .csv

In [65]:
df_total_original.to_csv("df_total_original.csv", index = False)